In [1]:
import tensorflow as tf
import numpy as np
import scipy as sp
import sklearn as skl
import matplotlib.pyplot as plt
import pandas as pd
import random
import os
import glob
import librosa as lba
import librosa.display as lid
from pathlib import Path
from tensorflow import keras
from keras import layers
from keras import models
from keras import regularizers
from keras.constraints import max_norm
from sklearn.model_selection import train_test_split
#from sklearn.cluster import KMeans
from datetime import datetime, date, timedelta
from numpy.random import default_rng
from keras import backend as K
from keras.callbacks import ModelCheckpoint

WORKING_DIRECTORY = 'C:/BirdCLEF23/'
os.chdir (WORKING_DIRECTORY)

SAVE = './models/264b v2.1/v644_264b.hdf5'
WEIGHTS  = './models/264b v2.1/v644_264b_weights.hdf5'
DIR_MELS = './mels/264b v2.1/'

LR = 0.001 #calculated by scheduler (epoch, lr)
DR = 0.2
DRA = 0.2

SR = 32000
iSR5 = SR*5

SEED = 2276
rng = default_rng (SEED)
np.random.seed (SEED)
random.seed (SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
tf.random.set_seed (SEED)

fpLog  = Path ('./logs/log sc v6.4.4 264b.txt')

with open (fpLog, 'w') as flog:
    print ('Start SoundClassifier v6.4.4 264b at : ', datetime.now(), file = flog)

#lmd = [dname.replace ('\\', '/').replace ('//', '/') for dname in glob.glob (f'{DIR_MELS}/*')]

#ltrain = []; lval = []

#for dname in lmd:
#    _lm_  = [fname.replace ('\\', '/').replace ('//', '/') for fname in glob.glob (f'{dname}/*.npy')]
#    _ltr_, _lvl_ = train_test_split (_lm_, test_size = 0.1, random_state = SEED)
#    ltrain = ltrain + _ltr_; lval = lval + _lvl_

lclasses = sorted (os.listdir (DIR_MELS))
tnclasses = tf.constant (lclasses)
NUM_CLASSES = len (lclasses)
NUM_Y_STRIPES = 64
NUM_X_STRIPES = 200
BSIZE = 64

#dfm = pd.read_html ('./config 264b/mels.html', encoding = 'UTF-8')[0]

#ltrain = []; lval = []

#for bird in lclasses:
#    dfs = dfm.loc [(dfm['bird'] == bird)]
#    _lm_ = dfs.fname.tolist() 
#    _ltr_, _lvl_ = train_test_split (_lm_, test_size = 0.1, random_state = SEED)
#    ltrain = ltrain + _ltr_; lval = lval + _lvl_

##TRNSAMS = './config 264b v2.0/registry_trn v1.1.html'
##VALSAMS = './config 264b v2.0/registry_val v1.1.html'

##dtrn = pd.read_html (TRNSAMS, encoding = 'UTF-8')[0]
##dval = pd.read_html (VALSAMS, encoding = 'UTF-8')[0]

##ltrain = dtrn.sname.tolist(); lval = dval.sname.tolist()

REGISTRY_MELS = './config 264b v2.1/registry_mels.html'

drm = pd.read_html (REGISTRY_MELS, encoding = 'UTF-8')[0]
ltrain = drm.loc[drm['split'] == 'TRN'].sname.tolist()
lval   = drm.loc[drm['split'] == 'VAL'].sname.tolist()

# Data Generator

def data_generator (lfiles, batch_size = 32):

    i = 0
    rng.shuffle (lfiles)

#    dfr = pd.read_html ('./config/2lbl marked up.html', encoding = 'UTF-8')[0]
#    l2l = sorted (dfr.fname.tolist())
#    dfr = dfr.set_index (['fname'])

    while True:
        if i*batch_size >= len(lfiles):  # This loop is used to run the generator indefinitely.
            i = 0
            rng.shuffle (lfiles)
        else:
            file_chunk = lfiles[i*batch_size:(i+1)*batch_size] 
            data = []
            labels = []

            for file in file_chunk:
                spec = np.load (file, allow_pickle = True, fix_imports = True, encoding = 'latin1')
                spec = np.abs (spec + 80)
                spec = spec/np.amax (spec)
                spec.resize ((128, 400))
                #data.append (spec.reshape (128, 400)) 
                data.append (spec) 
                strname = tf.compat.as_str_any (file)

                label = tf.constant (strname.split ('/')[-2]) 
                labels.append (tnclasses.numpy().tolist().index(label.numpy()))

#                if strname not in l2l:
#                    label = tf.constant (strname.split ('/')[-2]) 
#                    labels.append (tnclasses.numpy().tolist().index(label.numpy()))
#                else:
#                    labels.append (dfr.loc[(strname), ['P']].array[0])
 
            data = tf.convert_to_tensor (np.asarray(data).reshape (-1, 128, 400))
            labels = np.asarray (labels)
            labels = tf.one_hot (labels, NUM_CLASSES)
            yield data, labels
            i = i + 1

# Datasets

dstrain = tf.data.Dataset.from_generator (
    data_generator, 
    args = [ltrain, BSIZE], 
    output_types = (tf.float32, tf.float32),
    output_shapes = ((None, 128, 400), (None, NUM_CLASSES))
    )

dsval = tf.data.Dataset.from_generator (
    data_generator, 
    args = [lval, BSIZE], 
    output_types = (tf.float32, tf.float32),
    output_shapes = ((None, 128, 400), (None, NUM_CLASSES))
    )

# replace relu на swish = x * backend.sigmoid(x) 

input = layers.Input (shape = (128, 400))

ktn1 = layers.Reshape ((NUM_X_STRIPES, 400//NUM_X_STRIPES, 128, 1))(input) #(None, 200, 2, 128, 1)
ktn1 = layers.Conv2D (10, kernel_size = (2, 1), strides = (1, 1), kernel_regularizer = regularizers.l2(0.0001), activation = "selu") (ktn1) #(None, 100, 1, 128, 12)
ktn1 = layers.BatchNormalization()(ktn1)
ktn1 = layers.Reshape ((200, 128, 10))(ktn1) #(None, 100, 128, 12)

ktn1 = layers.Dropout (DRA)(ktn1)

ktn1 = layers.Conv1D (10, 40, strides = 1, padding = 'same', kernel_regularizer = regularizers.l2(0.0001), activation = "selu")(ktn1) #(None, 100, 128, 12)
ktn1 = layers.MaxPool2D (pool_size = (1, 2), strides = (1, 2), padding = 'same')(ktn1) #(None, 100, 64, 12)
ktn1 = layers.BatchNormalization()(ktn1)

ktn1 = layers.Dropout (DR)(ktn1)

ktn1 = layers.Conv1D (10, 16, strides = 4, padding = 'same', kernel_regularizer = regularizers.l2(0.0001), activation = "selu")(ktn1) #(None, 100, 16, 12)
ktn1 = layers.MaxPool2D (pool_size = (1, 2), strides = (1, 2), padding = 'same')(ktn1) #(None, 100, 8, 12)
ktn1 = layers.BatchNormalization()(ktn1)

ktn1 = layers.Dropout (DRA)(ktn1)

ktn1 = layers.Conv1D (5, 2, strides = 1, padding = 'same', kernel_regularizer = regularizers.l2(0.0001), activation = "selu")(ktn1) #(None, 100, 8, 6)
ktn1 = layers.BatchNormalization()(ktn1)

ktn1 = layers.Dropout (DR)(ktn1)

ktn1 = layers.Conv1D (3, 2, strides = 2, padding = 'same', kernel_regularizer = regularizers.l2(0.0001), activation = "selu") (ktn1) #(None, 100, 4, 4)

ktn1 = layers.Flatten()(ktn1)
ktn1 = layers.BatchNormalization()(ktn1)

ktn2 = layers.Reshape ((NUM_Y_STRIPES, 128//NUM_Y_STRIPES, 400, 1))(input) #(None, 64, 2, 400, 1)
ktn2 = layers.Conv2D (20, kernel_size = (2, 1), strides = (1, 1), kernel_regularizer = regularizers.l2(0.0001), activation = "selu") (ktn2) #(None, 32, 1, 400, 12)
ktn2 = layers.Reshape ((64, 400, 20))(ktn2) #(None, 32, 400, 12)
ktn2 = layers.BatchNormalization()(ktn2)

#ktn2 = layers.Dropout (DR)(ktn2)

#ktn2 = layers.Reshape ((128, 400, 1))(input) #(None, 128, 400, 1)
#ktn2 = layers.Conv1D (10, 120, strides = 1,  padding = 'same', kernel_regularizer = regularizers.l2(0.0001), activation = "selu") (ktn2) #(None, 128, 400, 10)
#ktn2 = layers.MaxPool2D (pool_size = (1, 4), strides = (1, 4), padding = 'same')(ktn2) #(None, 128, 100, 10)
#ktn2 = layers.BatchNormalization()(ktn2)

ktn2 = layers.Dropout (DR)(ktn2)

ktn2 = layers.Conv1D (20, 120, strides = 1, padding = 'same', kernel_regularizer = regularizers.l2(0.0001), activation = "selu") (ktn2) #(None, 32, 400, 12)
ktn2 = layers.MaxPool2D (pool_size = (1, 4), strides = (1, 4), padding = 'same')(ktn2) #(None, 32, 100, 12)
ktn2 = layers.BatchNormalization()(ktn2)

ktn2 = layers.Dropout (DRA)(ktn2)

ktn2 = layers.Conv1D (20, 30, strides = 1, padding = 'same', kernel_regularizer = regularizers.l2(0.0001), activation = "selu") (ktn2) #(None, 32, 100, 12)
ktn2 = layers.MaxPool2D (pool_size = (1, 4), strides = (1, 4), padding = 'same')(ktn2) #(None, 32, 25, 12)
ktn2 = layers.BatchNormalization()(ktn2)

ktn2 = layers.Dropout (DRA)(ktn2)

ktn2 = layers.Conv1D (20, 8, strides = 1, padding = 'same', kernel_regularizer = regularizers.l2(0.0001), activation = "selu") (ktn2) #(None, 32, 25, 12)
ktn2 = layers.MaxPool2D (pool_size = (1, 2), strides = (1, 2))(ktn2) #(None, 32, 12, 12)
ktn2 = layers.BatchNormalization()(ktn2)

ktn2 = layers.Dropout (DR)(ktn2)

ktn2 = layers.Conv1D (10, 2, strides = 1, padding = 'same', activation = "selu") (ktn2) #(None, 32, 12, 6)

ktn2 = layers.Flatten()(ktn2)
ktn2 = layers.BatchNormalization()(ktn2)

ktnu = layers.Concatenate()([ktn1, ktn2])
ktnu = layers.Dropout (DR)(ktnu)
ktnu = layers.Dense (2100, activation = "selu")(ktnu)
ktnu = layers.BatchNormalization()(ktnu)
ktnu = layers.Dropout (DR)(ktnu)
ktnu = layers.Dense (800, activation = "relu")(ktnu)
ktnu = layers.BatchNormalization()(ktnu)
ktnu = layers.Dropout (DR)(ktnu)
out = layers.Dense (NUM_CLASSES, activation = "softmax")(ktnu)

# Building model:
model = models.Model (inputs = input, outputs = out)    
model.compile (optimizer = keras.optimizers.Adam (lr = LR), loss = 'categorical_crossentropy', metrics = ['accuracy'])

with open (fpLog, 'a') as flog:
    model.summary (print_fn = lambda x: flog.write(x + '\n'))

# Training the model:

checkpoint = ModelCheckpoint (
    WEIGHTS, 
    monitor = 'loss', 
    verbose = 1,
    WEIGHTS_weights_only = True,
    WEIGHTS_best_only = True, 
    mode ='auto', 
#    WEIGHTS_freq = BSIZE + 1
    )

try:
    model.load_weights (WEIGHTS)
except ValueError as error:
    with open (fpLog, 'a') as flog:
        print ('Value error : ', error, file = flog)
    pass
except OSError as error:
    with open (fpLog, 'a') as flog:
        print ('OS error : ', error, file = flog)
    pass

num_tsteps = int(np.ceil (len (ltrain)/BSIZE))
num_vsteps = int(np.ceil (len (lval)/BSIZE))

with open (fpLog, 'a') as flog:
    print (f'steps_per_epoch: {num_tsteps}', file = flog)
    print (f'validation_steps: {num_vsteps}',file = flog)

model.save (SAVE)

#def scheduler (epoch, lr):
#    if epoch == 0: 
#        return 0.001
#    elif epoch in [1,2,3,4,5]:
#        return 0.0001
#    else:
#        return 0.00001
    
def scheduler (epoch, lr):
    if epoch in [0,1]: 
        return 0.0001
    else:
        return 0.00001
    
learnrate = tf.keras.callbacks.LearningRateScheduler (scheduler)

model.fit (
    dstrain, 
    validation_data = dsval, 
    steps_per_epoch = num_tsteps,
    validation_steps = num_vsteps, 
    epochs = 30,
    verbose = 1, 
    callbacks = [checkpoint, learnrate]
#    callbacks = [checkpoint]
    )



C:\Users\stani\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/30
3705/3705 [==============================] - ETA: 0s - loss: 0.0181 - accuracy: 0.9997 
Epoch 1: saving model to ./models/264b v2.1\v644_264b_weights.hdf5
3705/3705 [==============================] - 41144s 11s/step - loss: 0.0181 - accuracy: 0.9997 - val_loss: 0.6755 - val_accuracy: 0.8523 - lr: 1.0000e-04
Epoch 2/30
3705/3705 [==============================] - ETA: 0s - loss: 0.0176 - accuracy: 0.9994 
Epoch 2: saving model to ./models/264b v2.1\v644_264b_weights.hdf5
3705/3705 [==============================] - 40753s 11s/step - loss: 0.0176 - accuracy: 0.9994 - val_loss: 0.6125 - val_accuracy: 0.8729 - lr: 1.0000e-04
Epoch 3/30
3705/3705 [==============================] - ETA: 0s - loss: 0.0344 - accuracy: 0.9939 
Epoch 3: saving model to ./models/264b v2.1\v644_264b_weights.hdf5
3705/3705 [==============================] - 41433s 11s/step - loss: 0.0344 - accuracy: 0.9939 - val_loss: 0.5742 - val_accuracy: 0.8807 - lr: 1.0000e-05
Epoch 4/30
3705/3705 [==================